In [1]:
import pandas as pd

train = pd.read_csv("train.csv", index_col=0)
test = test_pd = pd.read_csv("test.csv", index_col=0)


# print(train['Attrition'].value_counts())
# 处理Attrition字段
train["Attrition"] = train["Attrition"].map(lambda x: 1 if x == "Yes" else 0)


# 查看数据是否有空值
# print(train.isna().sum())

# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(["EmployeeNumber", "StandardHours"], axis=1)
test = test.drop(["EmployeeNumber", "StandardHours"], axis=1)


from sklearn.preprocessing import LabelEncoder

# 对于分类特征进行特征值编码
attr = [
    "Age",
    "BusinessTravel",
    "Department",
    "Education",
    "EducationField",
    "Gender",
    "JobRole",
    "MaritalStatus",
    "Over18",
    "OverTime",
]
lbe_list = []
for feature in attr:
    lbe = LabelEncoder()
    train[feature] = lbe.fit_transform(train[feature])
    test[feature] = lbe.transform(test[feature])
    lbe_list.append(lbe)

# print(train)

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    train.drop("Attrition", axis=1), train["Attrition"], test_size=0.2, random_state=42
)

X_train.head()

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
user_id,,,,,,,,,,,,,,,,,,,,,
434,13,2,1274,1,9,0,1,1,3,1,...,4,4,1,13,6,4,13,8,0,8
1455,22,2,1322,1,2,3,1,1,3,1,...,3,4,0,8,2,3,2,2,2,2
1185,25,2,1291,1,15,1,1,1,3,1,...,4,1,1,14,3,3,14,10,6,11
1115,16,2,479,1,7,3,3,1,1,1,...,3,3,0,1,4,1,1,0,0,0
269,18,2,1403,1,6,2,1,1,4,1,...,3,3,1,16,4,3,15,13,10,11


In [3]:
from sklearn.preprocessing import MinMaxScaler

# feature_range=(0, 1) 归一化后的范围
mms = MinMaxScaler(feature_range=(0, 1))
X_train = mms.fit_transform(X_train)
X_valid = mms.fit_transform(X_valid)
test = mms.fit_transform(test)

In [4]:
from sklearn.svm import SVC, LinearSVC

# 非线性
model = SVC(
    kernel="rbf",
    gamma="auto",
    max_iter=1000,
    random_state=33,
    verbose=True,
    tol=1e-5,
    cache_size=50000,
)


# 线性
model = LinearSVC(max_iter=1000, random_state=33, verbose=True,)


model.fit(X_train, y_train)
predict = model.predict(test)

# predict = model.predict_proba(test)[:, 1]

test_pd["Attrition"] = predict

# 转化为二分类输出
# test_pd['Attrition']=test_pd['Attrition'].map(lambda x:1 if x>=0.5 else 0)

test_pd[["Attrition"]].to_csv("submit_svc.csv")

[LibLinear]